# LoRA sur AMD Radeon 680M

## Transformation corpus au format ChatML

In [1]:
import json
import os
import random
import re
import sys
import time
from typing import Any, Dict, List, Optional



In [2]:
import os, sys
# --- Posez les variables AVANT d'importer torch ---
os.environ["HSA_OVERRIDE_GFX_VERSION"] = "10.3.0"
os.environ["AMD_SERIALIZE_KERNEL"] = "3"
os.environ["PYTORCH_SDPA_ENABLE_HEURISTIC"] = "0"
os.environ["PYTORCH_SDPA_ALLOW_MATH"] = "1"
os.environ["PYTORCH_SDPA_ENABLE_FLASH"] = "0"
os.environ["PYTORCH_SDPA_ENABLE_MEM_EFFICIENT"] = "0"
os.environ["HIP_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_HIP_ALLOC_CONF"]="expandable_segments:True"

import torch
print("torch.version.hip:", torch.version.hip)
print("env HSA_OVERRIDE_GFX_VERSION:", os.environ.get("HSA_OVERRIDE_GFX_VERSION"))
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device 0:", torch.cuda.get_device_name(0))

torch.version.hip: 6.1.40091-a8dbc0c19
env HSA_OVERRIDE_GFX_VERSION: 10.3.0
CUDA available: True
Device 0: AMD Radeon 680M


In [3]:
x = torch.randn(2, 128, 128, device="cuda")
y = x @ x.transpose(-1, -2)
print("OK matmul:", y.shape, y.device)


OK matmul: torch.Size([2, 128, 128]) cuda:0


/tmp/ipykernel_3687/745649980.py:1: UserWarning: expandable_segments not supported on this platform (Triggered internally at /pytorch/c10/hip/HIPAllocatorConfig.h:29.)
  x = torch.randn(2, 128, 128, device="cuda")


In [4]:
import transformers
print(torch.__version__)
print(torch.__file__)
print(transformers.__version__)

2.6.0+rocm6.1
/usr/local/lib/python3.10/dist-packages/torch/__init__.py
4.57.0


In [5]:
import transformers, peft, accelerate
print(transformers.__version__)  # → 4.45.x (ou +)
print(peft.__version__)          # → 0.12.x (ou +)
print(accelerate.__version__)    # → 0.33.x (ou +)


4.57.0
0.13.2
1.10.1


In [6]:
def load_data(path: str) -> List[str]:
    """Charge les données ancien format."""    
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue  # ignore les lignes vides
            try:
                obj = json.loads(line)
                data.append(obj)
            except json.JSONDecodeError as e:
                print(f"⚠️ Ligne {i} ignorée (erreur JSON) : {e}")
                continue
    return data


def save_data(path: str, data: List) -> None:
    # Sauvegarde dans un fichier au format JSON Lines
    with open(path, "w", encoding="utf-8") as fichier:
        for objet in data:
            fichier.write(json.dumps(objet, ensure_ascii=False) + "\n")



In [7]:
import json

old_train = load_data("data/old_train.jsonl")

chatml_train = []
for e in old_train:
    contenu = e.get("contenu") or ""
    user_text = (
        "Quelles sont les expressions clés contenues à l'identique dans ce texte : "
        + str(contenu)
    )

    # ⚠️ CIBLE D'APPRENTISSAGE : une CHAÎNE contenant un tableau JSON
    # Exemple: '["accords de Bretton Woods","dollar américain"]'
    target_list = e.get("expressions_clefs") or []
    assistant_text = json.dumps(list(map(str, target_list)), ensure_ascii=False)

    ne = {
        "messages": [
            {
                "role": "system",
                "content": (
                    "Vous êtes un extracteur d'expressions clés. Répondez UNIQUEMENT par un tableau JSON de chaînes, "
                    "sans commentaire. Incluez UNIQUEMENT les expressions, dates et lieux remarquables, évènements, "
                    "qui apparaissent à l'identique dans le texte."
                ),
            },
            {"role": "user", "content": user_text},
            {"role": "assistant", "content": assistant_text},  # <- str, mais c'est un tableau JSON sérialisé
        ]
    }
    chatml_train.append(ne)

# Écrire le JSONL final
save_data("data/train.jsonl", chatml_train)

# Vérif rapide : la cible est bien une chaîne qui ressemble à un tableau JSON
print(chatml_train[0]["messages"][-1]["content"])


["théorème d'incomplétude", "vérité mathématique", "limites de la raison humaine", "récursivité", "auto-référence"]


In [8]:

old_test = load_data("data/old_test.jsonl")

chatml_test = []
for e in old_test:
    contenu = e.get("contenu") or ""
    user_text = (
        "Quelles sont les expressions clés contenues à l'identique dans ce texte : "
        + str(contenu)
    )

    # ⚠️ CIBLE D'APPRENTISSAGE : une CHAÎNE contenant un tableau JSON
    # Exemple: '["accords de Bretton Woods","dollar américain"]'
    target_list = e.get("expressions_clefs") or []
    assistant_text = json.dumps(list(map(str, target_list)), ensure_ascii=False)

    ne = {
        "messages": [
            {
                "role": "system",
                "content": (
                    "Vous êtes un extracteur d'expressions clés. Répondez UNIQUEMENT par un tableau JSON de chaînes, "
                    "sans commentaire. Incluez UNIQUEMENT les expressions, dates et lieux remarquables, évènements, "
                    "qui apparaissent à l'identique dans le texte."
                ),
            },
            {"role": "user", "content": user_text},
            {"role": "assistant", "content": assistant_text},  # <- str, mais c'est un tableau JSON sérialisé
        ]
    }
    chatml_test.append(ne)

# Écrire le JSONL final
save_data("data/test.jsonl", chatml_test)

# Vérif rapide : la cible est bien une chaîne qui ressemble à un tableau JSON
print(chatml_test[0]["messages"][-1]["content"])


⚠️ Ligne 1027 ignorée (erreur JSON) : Invalid control character at: line 1 column 190 (char 189)
["droits humains", "affaires internationales", "violations des droits", "normes des droits", "promotion des droits"]


## Chargement dans un dataset

In [9]:
from datasets import load_dataset, DatasetDict
ds = load_dataset("json", data_files="data/train.jsonl")  # sans features
print(ds)  # doit montrer messages: list<struct<role: string, content: string>>

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 10000
    })
})


In [10]:
print(ds["train"].features)
print(ds["train"][0])


{'messages': [{'role': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}]}
{'messages': [{'role': 'system', 'content': "Vous êtes un extracteur d'expressions clés. Répondez UNIQUEMENT par un tableau JSON de chaînes, sans commentaire. Incluez UNIQUEMENT les expressions, dates et lieux remarquables, évènements, qui apparaissent à l'identique dans le texte."}, {'role': 'user', 'content': "Quelles sont les expressions clés contenues à l'identique dans ce texte : Les mathématiques de Gödel sont fascinantes et complexes. Son théorème d'incomplétude a profondément influencé la logique et les fondements des mathématiques. Il démontre qu'il existe des propositions mathématiques qui ne peuvent être ni prouvées ni réfutées au sein d'un système formel donné. Ce résultat remet en question la notion même de vérité mathématique et soulève des questions sur les limites de la raison humaine. En outre, Gödel explore des concepts tels que la récursivité et l'auto-référence, qui s

In [11]:
# --- 0) Paramètres rapides ---
ASSISTANT_TAG = "<|assistant|>:"   # balise devant la réponse
MAX_LEN = 512                      # mettez 384 ou 256 si la VRAM est juste
NUM_PROC = 4                       # parallel tokenization
USE_PACKING = True                 # True = moins de padding -> plus rapide

from datasets import Dataset, DatasetDict
from typing import Dict, Any, List
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "models/phi4"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, local_files_only=True)

# Votre ds actuel (déjà chargé) :
# ds = load_dataset("json", data_files="data/train.jsonl")
assert isinstance(ds, DatasetDict) and "train" in ds

# --- 1) messages -> text (chaîne plat) ---
def messages_to_text(ex: Dict[str, Any]) -> Dict[str, str]:
    msgs = ex.get("messages", [])
    # Formate: <|role|>: content\n...
    text = "\n".join(f"<|{m.get('role','user')}|>: {m.get('content','')}" for m in msgs)
    return {"text": text}

ds_txt = ds.map(messages_to_text, num_proc=NUM_PROC)
print(ds_txt)

# --- 2) Tokenisation + labels assistant-only pré-calculés ---
# (labels = -100 partout sauf après la DERNIÈRE occurrence de ASSISTANT_TAG)
assert 'tokenizer' in globals(), "Créez le tokenizer avant (AutoTokenizer.from_pretrained(...))."
tpl_ids: List[int] = tokenizer.encode(ASSISTANT_TAG, add_special_tokens=False)

def tok_and_mask(batch):
    t = tokenizer(batch["text"], truncation=True, max_length=MAX_LEN)
    labels = []
    for ids, attn in zip(t["input_ids"], t["attention_mask"]):
        lab = [-100] * len(ids)
        last = -1
        for j in range(0, len(ids) - len(tpl_ids) + 1):
            if ids[j:j+len(tpl_ids)] == tpl_ids:
                last = j
        if last >= 0:
            start = last + len(tpl_ids)
            end = max(i for i,a in enumerate(attn) if a == 1) + 1  # jusqu'au dernier token non-pad
            lab[start:end] = ids[start:end]
        labels.append(lab)
    t["labels"] = labels
    return t

remove_cols = [c for c in ds_txt["train"].column_names if c != "text"]
ds_tok = ds_txt.map(tok_and_mask, batched=True, num_proc=NUM_PROC, remove_columns=remove_cols)
print(ds_tok)

# --- 3) (Optionnel) Packing constant-length pour réduire le padding ---
def pack_constant_length(ds_split, max_len: int):
    big_ids, big_mask, big_lab = [], [], []
    for rec in ds_split:
        big_ids.extend(rec["input_ids"])
        big_mask.extend(rec["attention_mask"])
        big_lab.extend(rec["labels"])
    L = min(len(big_ids), len(big_mask), len(big_lab))
    L = (L // max_len) * max_len
    big_ids, big_mask, big_lab = big_ids[:L], big_mask[:L], big_lab[:L]
    chunks = []
    for i in range(0, L, max_len):
        chunks.append({
            "input_ids": big_ids[i:i+max_len],
            "attention_mask": big_mask[i:i+max_len],
            "labels": big_lab[i:i+max_len],
        })
    return Dataset.from_list(chunks)

if USE_PACKING:
    ds_packed = {}
    for k in ds_tok.keys():
        ds_packed[k] = pack_constant_length(ds_tok[k], MAX_LEN)
    ds_packed = DatasetDict(ds_packed)
    print(ds_packed)
    train_dataset_final = ds_packed["train"]
    eval_dataset_final  = ds_packed.get("validation")
else:
    train_dataset_final = ds_tok["train"]
    eval_dataset_final  = ds_tok.get("validation")

len(train_dataset_final), train_dataset_final[0].keys()


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'text'],
        num_rows: 10000
    })
})


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4890
    })
})


(4890, dict_keys(['input_ids', 'attention_mask', 'labels']))


# ⚡ Section: Pipeline rapide (pré-tokenisation + labels + packing)
Cette section ajoute un flux **plus rapide** et **plus stable** pour l'entraînement **LoRA classique** :
- Pré-calcul **une seule fois** des `labels` *assistant-only* (plus de scan par batch).
- **Packing** de séquences en blocs de longueur fixe pour réduire le padding (optionnel).
- Variables claires : `ds_raw` → `ds_txt` → `ds_tok` → `ds_packed`.
- Exemple de `Trainer` qui réutilise `model` et `args` déjà définis dans votre notebook.


In [12]:

# --- Paramètres de cette section ---
ASSISTANT_TAG = "<|assistant|>:"      # balise qui précède les réponses
MAX_LEN = globals().get("MAX_LEN", 512)  # fallback si non défini ailleurs
NUM_PROC = 4                            # ajustez selon vos CPU
USE_PACKING = True                      # True = réduit padding → accélère
model_path = "models/phi4"

from datasets import Dataset, DatasetDict
from typing import List, Dict, Any

# On suppose qu'un objet `ds` existe déjà avec au moins ds["train"]
# Sinon, adaptez ici en chargeant vos données dans ds_raw.
try:
    ds_raw = ds
except NameError:
    raise RuntimeError("Aucun dataset `ds` trouvé dans le notebook. Définissez `ds` (DatasetDict) avant d'exécuter cette section.")

assert isinstance(ds_raw, (dict, DatasetDict)) and "train" in ds_raw, "Le dataset `ds` doit être un DatasetDict avec une clé 'train'."


In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, DataCollatorWithPadding, Trainer

# ----- 0) Libérer la VRAM si vous sortez d’un OOM -----
def hard_free_gpu(*names):
    import gc
    G = globals()
    for n in names or ("trainer","trainer_fast","model","peft_model","lora","optimizer","scheduler","dataloader"):
        if n in G:
            obj = G[n]
            try:
                if hasattr(obj, "model") and hasattr(obj.model, "to"): obj.model.to("cpu")
            except: pass
            try:
                if hasattr(obj, "to"): obj.to("cpu")
            except: pass
            try: del G[n]
            except: pass
    gc.collect()
    if torch.cuda.is_available():
        try: torch.cuda.empty_cache()
        except: pass
        if hasattr(torch.cuda, "ipc_collect"):
            try: torch.cuda.ipc_collect()
            except: pass

hard_free_gpu()  # lancez-la juste après un OOM

# ----- 1) Charger modèle & tokenizer (BF16) directement sur GPU -----
model_path = "models/phi4"
tok = AutoTokenizer.from_pretrained(model_path, use_fast=True, local_files_only=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
    tok.pad_token_id = tok.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map={"": 0},      # place directement chaque sous-module sur cuda:0
    dtype=torch.bfloat16,    # (remplace torch_dtype=... déprécié)
    low_cpu_mem_usage=True,
    local_files_only=True,
)
# ROCm: évite SDPA “flash/mem-efficent” potentiellement lourd/instable
try:
    model.config.attn_implementation = "eager"
except Exception:
    pass

model.config.use_cache = False  # utile en train même sans checkpointing

# ----- 2) Appliquer LoRA sur les bons modules -----
from peft import LoraConfig, get_peft_model, TaskType
lora = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"],
)
model = get_peft_model(model, lora)
model.print_trainable_parameters()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 8,912,896 || all params: 3,844,934,656 || trainable%: 0.2318


In [14]:

# 1) Uniformiser en 'text': messages -> string, ou conserver si 'text' déjà présent.
def messages_to_text(ex: Dict[str, Any]) -> str:
    # si ex["messages"] existe: concatène "<|role|>: content"
    msgs = ex.get("messages")
    if isinstance(msgs, list) and msgs and isinstance(msgs[0], dict):
        return "\n".join(f"<|{m.get('role','user')}|>: {m.get('content','')}" for m in msgs)
    # sinon, si 'text' existe déjà
    if "text" in ex and isinstance(ex["text"], str):
        return ex["text"]
    # fallback: joindre toutes les valeurs str
    return " ".join(str(v) for v in ex.values() if isinstance(v, str))

def add_text_column(ds_split):
    return ds_split.map(lambda ex: {"text": messages_to_text(ex)}, num_proc=NUM_PROC)

ds_txt = {}
for k in ds_raw.keys():
    ds_txt[k] = add_text_column(ds_raw[k])
ds_txt = DatasetDict(ds_txt)
print(ds_txt)


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'text'],
        num_rows: 10000
    })
})


In [15]:

# 2) Tokenisation + pré-calcul des 'labels' assistant-only
from transformers import PreTrainedTokenizerBase
assert 'tokenizer' in globals(), "Le tokenizer doit exister (variable `tokenizer`)."

tpl_ids: List[int] = tokenizer.encode(ASSISTANT_TAG, add_special_tokens=False)

def tok_and_mask(batch):
    t = tokenizer(batch["text"], truncation=True, max_length=MAX_LEN)
    labels = []
    for ids, attn in zip(t["input_ids"], t["attention_mask"]):
        lab = [-100] * len(ids)
        last = -1
        # chercher la dernière occurrence du template
        for j in range(0, len(ids) - len(tpl_ids) + 1):
            if ids[j:j+len(tpl_ids)] == tpl_ids:
                last = j
        if last >= 0:
            start = last + len(tpl_ids)
            # fin = dernier token non-pad
            end = max(i for i, a in enumerate(attn) if a == 1) + 1
            lab[start:end] = ids[start:end]
        labels.append(lab)
    t["labels"] = labels
    return t

remove_cols = [c for c in ds_txt["train"].column_names if c != "text"]
ds_tok = {}
for k in ds_txt.keys():
    ds_tok[k] = ds_txt[k].map(tok_and_mask, batched=True, num_proc=NUM_PROC, remove_columns=remove_cols)
ds_tok = DatasetDict(ds_tok)
print(ds_tok)


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})


In [16]:

# 3) Packing constant-length (optionnel mais recommandé pour accélérer)
def pack_constant_length(ds_split, max_len: int):
    big_ids, big_mask, big_lab = [], [], []
    for rec in ds_split:
        big_ids.extend(rec["input_ids"])
        big_mask.extend(rec["attention_mask"])
        big_lab.extend(rec["labels"])
    # Taille minimale pour aligner labels/mask/ids
    L = min(len(big_ids), len(big_mask), len(big_lab))
    L = (L // max_len) * max_len
    big_ids, big_mask, big_lab = big_ids[:L], big_mask[:L], big_lab[:L]
    # Re-chunk
    from datasets import Dataset
    chunks = []
    for i in range(0, L, max_len):
        chunks.append({
            "input_ids": big_ids[i:i+max_len],
            "attention_mask": big_mask[i:i+max_len],
            "labels": big_lab[i:i+max_len],
        })
    return Dataset.from_list(chunks)

if USE_PACKING:
    ds_packed = {}
    for k in ds_tok.keys():
        ds_packed[k] = pack_constant_length(ds_tok[k], MAX_LEN)
    from datasets import DatasetDict
    ds_packed = DatasetDict(ds_packed)
    print(ds_packed)
    train_dataset_final = ds_packed["train"]
    eval_dataset_final  = ds_packed.get("validation")
else:
    train_dataset_final = ds_tok["train"]
    eval_dataset_final  = ds_tok.get("validation")


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4890
    })
})


In [ ]:

# 4) Trainer: on réutilise `model` et `args` s'ils existent déjà.
from transformers import TrainingArguments, DataCollatorWithPadding, Trainer

if "args" not in globals():
    args = TrainingArguments(
        output_dir="./checkpoints",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        num_train_epochs=1,
        learning_rate=2e-5,
        logging_steps=100,
        save_strategy="epoch",
        report_to="none",
        bf16=True,
        fp16=False,
        optim="adamw_torch",
        dataloader_num_workers=2,
        dataloader_pin_memory=False,
    )

collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

trainer_fast = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset_final,
    eval_dataset=eval_dataset_final,
    data_collator=collator,
)

print("Batches/train:", len(trainer_fast.get_train_dataloader()))



### Lancer l'entraînement rapide
Exécutez la cellule ci-dessous pour entraîner avec le pipeline optimisé.


In [ ]:

train_result = trainer_fast.train()
print(train_result)

if trainer_fast.eval_dataset is not None:
    metrics = trainer_fast.evaluate()
    print(metrics)

trainer_fast.save_model("./checkpoints-fast")
tokenizer.save_pretrained("./checkpoints-fast")


In [17]:
# === Évaluation avec barres de progression ===
import os, json, math, torch, warnings, glob
from pathlib import Path
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
from tqdm.auto import tqdm  # <- barre de progression Jupyter/terminal

MODEL_PATH = model_path if "model_path" in globals() else "models/phi4"
BASE_ADAPTER_DIR = "checkpoints_phi4_lora"  # <- votre dossier effectif
TEST_FILE = "data/test.jsonl"
MAX_LEN_EVAL = 1024
BATCH = 2

assert Path(TEST_FILE).exists(), f"{TEST_FILE} introuvable."
print(f"[i] Modèle : {MODEL_PATH}")
print(f"[i] Répertoire d’adaptateur (base) : {BASE_ADAPTER_DIR}")

def find_adapter_dir(root: str) -> str | None:
    if Path(root, "adapter_config.json").exists():
        return root
    cands = glob.glob(os.path.join(root, "**", "adapter_config.json"), recursive=True)
    return str(Path(cands[0]).parent) if cands else None

ADAPTER_DIR = find_adapter_dir(BASE_ADAPTER_DIR)
print(f"[i] Adaptateur résolu : {ADAPTER_DIR or '(aucun, modèle de base)'}")

tok = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True, local_files_only=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
    tok.pad_token_id = tok.eos_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    dtype=(torch.bfloat16 if torch.cuda.is_available() else torch.float32),
    low_cpu_mem_usage=True,
    local_files_only=True,
).to(device)

from peft import PeftModel
if ADAPTER_DIR:
    try:
        model = PeftModel.from_pretrained(model, ADAPTER_DIR, is_trainable=False)
        print("[i] Adaptateur LoRA chargé.")
        try:
            model = model.merge_and_unload()
            print("[i] Adaptateur fusionné (merge_and_unload) pour inférence.")
        except Exception:
            pass
    except Exception as e:
        warnings.warn(f"Échec du chargement de l’adaptateur : {e}")
else:
    warnings.warn("Aucun adaptateur détecté — utilisation du modèle de base.")

model.eval()
model.config.use_cache = True
try: model.config.attn_implementation = "eager"
except Exception: pass

# --------- Chargement + tokenisation avec barre de progression ---------
def join_chatml(msgs):
    return "\n".join(f"<|{m['role']}|> {m['content']}" for m in msgs)

ds_test = load_dataset("json", data_files=TEST_FILE, split="train")

def tok_fn(ex):
    text = join_chatml(ex["messages"])
    return tok(text, truncation=True, max_length=MAX_LEN_EVAL, padding=False)

# La lib `datasets` affiche déjà une barre, on renforce avec desc explicite :
tok_ds = ds_test.map(tok_fn, remove_columns=["messages"], desc="Tokenisation test")

if "labels" in tok_ds.column_names:
    tok_ds = tok_ds.remove_columns("labels")

def lm_collate(batch):
    padded = tok.pad(batch, padding=True, pad_to_multiple_of=8, return_tensors="pt")
    labels = padded["input_ids"].clone()
    if "attention_mask" in padded:
        labels[padded["attention_mask"] == 0] = -100
    else:
        labels[labels == tok.pad_token_id] = -100
    padded["labels"] = labels
    return padded

dl = DataLoader(tok_ds, batch_size=BATCH, shuffle=False, collate_fn=lm_collate)

# --------- Boucle d’évaluation avec barre de progression ---------
loss_sum, count = 0.0, 0
for batch in tqdm(dl, desc="Évaluation (loss/ppl)"):
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        loss = out.loss
        if not torch.isnan(loss):
            loss_sum += float(loss.item())
            count += 1

if count:
    mean_loss = loss_sum / count
    ppl = math.exp(mean_loss) if mean_loss < 20 else float("inf")
    print(f"[✓] Mean loss: {mean_loss:.4f} | Perplexity: {ppl:.2f}")
else:
    print("[!] Aucun batch valide pour la perte.")

# --------- Génération (pas de barre utile pour un seul exemple) ---------
example_msgs = ds_test[0]["messages"]
prompt = join_chatml(example_msgs[:-1])
inputs = tok(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    gen = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
    )

print("\n[Exemple de génération]")
print(tok.decode(gen[0], skip_special_tokens=True))


[i] Modèle : models/phi4
[i] Répertoire d’adaptateur (base) : checkpoints_phi4_lora
[i] Adaptateur résolu : checkpoints_phi4_lora


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[i] Adaptateur LoRA chargé.
[i] Adaptateur fusionné (merge_and_unload) pour inférence.


Generating train split: 0 examples [00:00, ? examples/s]

Tokenisation test:   0%|          | 0/1026 [00:00<?, ? examples/s]

Évaluation (loss/ppl):   0%|          | 0/513 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[✓] Mean loss: 2.4405 | Perplexity: 11.48

[Exemple de génération]
Vous êtes un extracteur d'expressions clés. Répondez UNIQUEMENT par un tableau JSON de chaînes, sans commentaire. Incluez UNIQUEMENT les expressions, dates et lieux remarquables, évènements, qui apparaissent à l'identique dans le texte.
Quelles sont les expressions clés contenues à l'identique dans ce texte : Les droits humains sont des principes fondamentaux qui garantissent la dignité et l'égalité de tous les individus, sans distinction. Dans le cadre des affaires internationales, le respect de ces droits est souvent au cœur des discussions diplomatiques. Les violations des droits humains peuvent conduire à des sanctions, des interventions ou des résolutions au sein d'organisations telles que l'ONU. Les traités internationaux, comme la Déclaration universelle des droits de l'homme, établissent des normes que les États doivent respecter. La promotion des droits humains est essentielle pour maintenir la paix et la sécur

In [5]:
# === Fusion LoRA -> HF, conversion GGUF f16 et quantification llama.cpp ===
import os, sys, shutil, subprocess
from pathlib import Path
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# --- chemins fixes ---
MODEL_PATH = "models/phi4"
ADAPTER_DIR = "checkpoints_phi4_lora"
LLAMA_CPP = "/workspace/llama.cpp"
CONVERT_SCRIPT = f"{LLAMA_CPP}/convert_hf_to_gguf.py"
QUANT_BIN = f"{LLAMA_CPP}/build/bin/llama-quantize"
QUANT_RUN = f"{LLAMA_CPP}/build/bin/llama-run"
OUT_DIR = Path("gguf_out/phi4_merged")
GGUF_OUTTYPE = "f16"
QUANTS = ["Q4_K_M", "Q5_K_M", "Q6_K"]

# --- chargement et fusion ---
print(f"[i] Chargement du modèle : {MODEL_PATH}")
tok = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True, local_files_only=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
    tok.pad_token_id = tok.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    dtype=torch.float32,
    device_map="cpu",
    low_cpu_mem_usage=True,
    local_files_only=True,
)

print(f"[i] Chargement adaptateur LoRA depuis {ADAPTER_DIR}")
model = PeftModel.from_pretrained(model, ADAPTER_DIR, is_trainable=False)
print("[i] Fusion (merge_and_unload)...")
model = model.merge_and_unload()

# --- sauvegarde HF fusionné ---
MERGED_DIR = OUT_DIR / "merged_hf"
MERGED_DIR.mkdir(parents=True, exist_ok=True)
model.save_pretrained(MERGED_DIR)
tok.save_pretrained(MERGED_DIR)
print(f"[✓] Modèle fusionné sauvegardé : {MERGED_DIR}")

# --- conversion HF → GGUF f16 ---
gguf_f16 = OUT_DIR / f"model-{GGUF_OUTTYPE}.gguf"
OUT_DIR.mkdir(parents=True, exist_ok=True)

subprocess.run([
    sys.executable, CONVERT_SCRIPT,
    "--outtype", GGUF_OUTTYPE,
    "--outfile", str(gguf_f16),
    str(MERGED_DIR)
], check=True)

print(f"[✓] Conversion GGUF f16 : {gguf_f16} ({gguf_f16.stat().st_size/1e6:.1f} MB)")

# --- quantification ---
for q in QUANTS:
    out_q = OUT_DIR / f"model-{q}.gguf"
    subprocess.run([QUANT_BIN, str(gguf_f16), str(out_q), q], check=True)
    print(f"[✓] Quantifié {q} -> {out_q} ({out_q.stat().st_size/1e6:.1f} MB)")

print("\n[Résumé sorties]")
for f in sorted(OUT_DIR.glob("model-*.gguf")):
    print(f" - {f.name:20s}  {f.stat().st_size/1e6:.1f} MB")

print("\n[Conseil] Test rapide :")
print(f"  {LLAMA_CPP}/build/bin/main -n 64 {OUT_DIR}/model-Q4_K_M.gguf 'Dites bonjour.' -n 64")


[i] Chargement du modèle : models/phi4


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[i] Chargement adaptateur LoRA depuis checkpoints_phi4_lora
[i] Fusion (merge_and_unload)...
[✓] Modèle fusionné sauvegardé : gguf_out/phi4_merged/merged_hf


INFO:hf-to-gguf:Loading model: merged_hf
INFO:hf-to-gguf:Model architecture: Phi3ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_factors_long.weight,  torch.float32 --> F32, shape = {48}
INFO:hf-to-gguf:rope_factors_short.weight, torch.float32 --> F32, shape = {48}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.float32 --> F16, shape = {3072, 200064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float32 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.float32 --> F16, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.float32 --> F16, shape = {3072, 16384}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.float32 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,  torch.float32 -->

[✓] Conversion GGUF f16 : gguf_out/phi4_merged/model-f16.gguf (7680.7 MB)


...


main: quantize time = 40877.65 ms
main:    total time = 40877.65 ms
[✓] Quantifié Q4_K_M -> gguf_out/phi4_merged/model-Q4_K_M.gguf (2491.9 MB)


main: build = 0 (unknown)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: quantizing 'gguf_out/phi4_merged/model-f16.gguf' to 'gguf_out/phi4_merged/model-Q5_K_M.gguf' as Q5_K_M
llama_model_loader: loaded meta data with 30 key-value pairs and 196 tensors from gguf_out/phi4_merged/model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:              phi3.rope.scaling.attn_factor f32              = 1.190238
llama_model_loader: - kv   2:                               general.type str              = model
llama_model_loader: - kv   3:                               general.name str              = Merged_Hf
llama_model_loader: - kv   4:                         general.size_label str              = 3.8B
llama_model_loader: - kv   5:                 


main: quantize time = 30997.43 ms
main:    total time = 30997.43 ms
[✓] Quantifié Q5_K_M -> gguf_out/phi4_merged/model-Q5_K_M.gguf (2848.1 MB)


main: build = 0 (unknown)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: quantizing 'gguf_out/phi4_merged/model-f16.gguf' to 'gguf_out/phi4_merged/model-Q6_K.gguf' as Q6_K
llama_model_loader: loaded meta data with 30 key-value pairs and 196 tensors from gguf_out/phi4_merged/model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:              phi3.rope.scaling.attn_factor f32              = 1.190238
llama_model_loader: - kv   2:                               general.type str              = model
llama_model_loader: - kv   3:                               general.name str              = Merged_Hf
llama_model_loader: - kv   4:                         general.size_label str              = 3.8B
llama_model_loader: - kv   5:                     


main: quantize time = 20994.98 ms
main:    total time = 20994.98 ms
[✓] Quantifié Q6_K -> gguf_out/phi4_merged/model-Q6_K.gguf (3155.6 MB)

[Résumé sorties]
 - model-Q4_K_M.gguf     2491.9 MB
 - model-Q5_K_M.gguf     2848.1 MB
 - model-Q6_K.gguf       3155.6 MB
 - model-f16.gguf        7680.7 MB

[Conseil] Test rapide :
  /workspace/llama.cpp/build/bin/main -m gguf_out/phi4_merged/model-Q4_K_M.gguf -p 'Dites bonjour.' -n 64


In [26]:
import subprocess
from pathlib import Path

LLAMA_RUN = "/workspace/llama.cpp/build/bin/llama-run"
OUT_DIR = Path("/workspace/lora_local/gguf_out/phi4_merged")
MODEL = f"{OUT_DIR}/model-Q4_K_M.gguf"

result = subprocess.run(
    [LLAMA_RUN, "-n", "64", MODEL, "Dites bonjour en un seul mot."],
    text=True,
    capture_output=True
)
print(result.stdout)

Salut!

